In [124]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [125]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [126]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Enunciado
Dado los acontecimientos en USA, deseamos obtener datos que nos den mayor información sobre las muertes de gente de raza negra por parte de oficiales de policía.

Para ello, tenemos un csv con información sobre las muertes por parte de oficiales de policía en USA para 2015 hasta 2017: 

(name, date, race, city, state)

Y otro csv con información sobre el porcentaje de pobreza en las ciudades de USA: 

(state, city, poverty_rate)

Se pide:

* a) Obtener las 10 ciudades con mayor diferencia entre el porcentaje de pobreza de la ciudad y el porcentaje de pobreza del estado en el que se encuentra esa ciudad. Por ejemplo si en la ciudad de Houston la pobreza es de 15.2 y la pobreza en Texas (el estado donde se encuentra Houston) es de 11.1, la diferencia es 4.1.(15 pts)
* b) Obtener la cantidad de muertes de gente de raza negra por parte de oficiales de policía, agrupada por estados que compartan el mismo nivel de pobreza redondeado al entero más cercano. Por ejemplo, si NJ tiene una pobreza de 10.33, AL una de 20.64 y AZ una de 10.44, NJ y AZ quedarían juntos representados por el nivel de pobreza de 10 y AL en otro grupo con el nivel 21. La salida debe tener el formato: (nivel_de_pobreza, total_de_muertes) (15 pts)
 
Resolver los puntos usando la API de RDDs de PySpark. (30 pts)


# CSV import

In [127]:
killing = sqlContext.read.csv('archive/PoliceKillingsUS.csv', header=True).rdd.map(lambda x: (x.name,x.date,x.race,x.city,x.state)).cache()
poverty = sqlContext.read.csv('archive/PercentagePeopleBelowPovertyLevel.csv', header=True).rdd\
.map(lambda x: (x[0],x[1],float(x.poverty_rate) if x.poverty_rate != '-' else 0)).cache()

In [128]:
killing.take(3)

[('Tim Elliot', '02/01/15', 'A', 'Shelton', 'WA'),
 ('Lewis Lee Lembke', '02/01/15', 'W', 'Aloha', 'OR'),
 ('John Paul Quintero', '03/01/15', 'H', 'Wichita', 'KS')]

In [129]:
poverty.take(3)

[('AL', 'Abanda CDP', 78.8),
 ('AL', 'Abbeville city', 29.1),
 ('AL', 'Adamsville city', 25.5)]

Empieza el ejercicio
* a) Obtener las 10 ciudades con mayor diferencia entre el porcentaje de pobreza de la ciudad y el porcentaje de pobreza del estado en el que se encuentra esa ciudad. Por ejemplo si en la ciudad de Houston la pobreza es de 15.2 y la pobreza en Texas (el estado donde se encuentra Houston) es de 11.1, la diferencia es 4.1.(15 pts)


In [130]:
#compruebo que solo haya un registro de Estado:ciudad
poverty.map(lambda x: ((x[0],x[1]),1)).reduceByKey(lambda x,y: x+y).takeOrdered(10,lambda x: -x[1])

[(('AL', 'Abanda CDP'), 1),
 (('AL', 'Abbeville city'), 1),
 (('AL', 'Adamsville city'), 1),
 (('AL', 'Addison town'), 1),
 (('AL', 'Akron town'), 1),
 (('AL', 'Alabaster city'), 1),
 (('AL', 'Albertville city'), 1),
 (('AL', 'Alexander City city'), 1),
 (('AL', 'Alexandria CDP'), 1),
 (('AL', 'Aliceville city'), 1)]

In [131]:
pobreza_estado = poverty.map(lambda x: (x[0],(x[2],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0],x[1][0]/x[1][1]))
pobreza_estado.takeOrdered(5,lambda x: -x[1])

[('MS', 26.884254143646384),
 ('AZ', 25.268070953436805),
 ('GA', 23.663636363636385),
 ('AR', 22.963216266173745),
 ('NM', 22.5076749435666)]

In [132]:
estado_ciudad_pobreza = poverty.map(lambda x: (x[0],(x[1],x[2])))
estado_ciudad_pobreza.take(3)

[('AL', ('Abanda CDP', 78.8)),
 ('AL', ('Abbeville city', 29.1)),
 ('AL', ('Adamsville city', 25.5))]

In [133]:
def fun(x):
    return (x[0],abs())

In [134]:
pobreza_estado.join(estado_ciudad_pobreza)\
    .map(lambda x: (x[1][1][0],(x[1][0],x[1][1][1])))\
    .map(lambda x: (x[0],__builtin__.abs(x[1][1] -x[1][0]))).takeOrdered(10,lambda x: -x[1])

[('Westview Circle CDP', 90.93627450980392),
 ('Middleburg CDP', 89.72760617760618),
 ('Waterview CDP', 89.72760617760618),
 ('Selz CDP', 87.86807980049875),
 ('Pioneer city', 87.7684523809524),
 ('Lynn CDP', 86.8122270742358),
 ('Kinbrae city', 86.25426356589146),
 ('The Ranch CDP', 86.25426356589146),
 ('Acme CDP', 85.17400318979267),
 ('Addy CDP', 85.17400318979267)]

b) Obtener la cantidad de muertes de gente de raza negra por parte de oficiales de policía, agrupada por estados que compartan el mismo nivel de pobreza redondeado al entero más cercano. Por ejemplo, si NJ tiene una pobreza de 10.33, AL una de 20.64 y AZ una de 10.44, NJ y AZ quedarían juntos representados por el nivel de pobreza de 10 y AL en otro grupo con el nivel 21. La salida debe tener el formato: (nivel_de_pobreza, total_de_muertes) (15 pts)

In [153]:
pobreza_estado = pobreza_estado.map(lambda x: (x[0],int(x[1])))
pobreza_estado.take(3)

[('AL', 20), ('AK', 18), ('AZ', 25)]

In [162]:
muertes_estado = killing.filter(lambda x: x[2] == 'B').map(lambda x: (x[4],1)).reduceByKey(lambda x,y: x+y)
muertes_estado.take(4)

[('PA', 21), ('TX', 46), ('OH', 30), ('LA', 32)]

In [163]:
#(estado,(pobreza_level,muertos_en_estado))
pobreza_estado.join(muertes_estado).map(lambda x: (x[1][0],x[1][1])).reduceByKey(lambda x,y: x+y).takeOrdered(100,lambda x: -x[0])

[(26, 8),
 (25, 5),
 (23, 23),
 (22, 52),
 (21, 3),
 (20, 36),
 (19, 109),
 (18, 11),
 (17, 64),
 (16, 70),
 (15, 13),
 (14, 61),
 (13, 49),
 (12, 47),
 (11, 22),
 (10, 25),
 (9, 7),
 (8, 13)]